# Panoptic segmentation evaluation using DETR



# Intallation

In [1]:
!git clone https://github.com/facebookresearch/detr.git
!conda install -c pytorch pytorch torchvision
!conda install cython scipy
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install git+https://github.com/cocodataset/panopticapi.git

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Cloning into 'detr'...
remote: Enumerating objects: 260, done.
remote: Total 260 (delta 0), reused 0 (delta 0), pack-reused 260
Receiving objects: 100% (260/260), 12.87 MiB | 8.68 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/bin/bash: conda: command not found
/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-s2kq2pdx
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-s2kq2pdx
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-linux_x86_64.whl size=304481 sha256=81fa909f1b7aca0ae686ac439e2790e372744e0b27668ab89c2f9c331e708958
  Stored in directory: /tmp/pip-ephem-wheel-cache-wxm6v12u/wheels/56/da/49/cb71a7c450b59588934077f431100c05fbde50646ee84a8d40
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0

In [2]:
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2

Cloning into 'detectron2'...
remote: Enumerating objects: 14634, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 14634 (delta 16), reused 31 (delta 11), pack-reused 14584
Receiving objects: 100% (14634/14634), 5.99 MiB | 9.28 MiB/s, done.
Resolving deltas: 100% (10566/10566), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/detectron2
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


# prepare dataset

In [3]:
%cd /content/

# download, decompress the data
!wget --directory-prefix=downloads http://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip
!wget --directory-prefix=downloads http://images.cocodataset.org/zips/train2017.zip
!wget --directory-prefix=downloads http://images.cocodataset.org/zips/val2017.zip
!wget --directory-prefix=downloads http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!wget --directory-prefix=downloads http://images.cocodataset.org/annotations/panoptic_annotations_trainval2017.zip



/content
--2022-12-11 05:54:20--  http://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip
Resolving calvin.inf.ed.ac.uk (calvin.inf.ed.ac.uk)... 129.215.202.26
Connecting to calvin.inf.ed.ac.uk (calvin.inf.ed.ac.uk)|129.215.202.26|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip [following]
--2022-12-11 05:54:20--  https://calvin.inf.ed.ac.uk/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip
Connecting to calvin.inf.ed.ac.uk (calvin.inf.ed.ac.uk)|129.215.202.26|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://calvin-vision.net/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip [following]
--2022-12-11 05:54:20--  https://calvin-vision.net/wp-content/uploads/data/cocostuffdataset/cocostuff-10k-v1.1.zip
Resolving calvin-vision.net (calvin-vision.n

In [4]:
!mkdir -p /content/datasets
!mkdir -p /content/datasets/coco

!unzip -d /content/datasets/coco/ downloads/annotations_trainval2017.zip
!unzip -d /content/datasets/coco/ downloads/panoptic_annotations_trainval2017.zip

!unzip -d /content/datasets/coco/ downloads/train2017.zip
!unzip -d /content/datasets/coco/ downloads/val2017.zip
!unzip -d /content/datasets/coco/ /content/datasets/coco/annotations/panoptic_train2017.zip
!unzip -d /content/datasets/coco/ /content/datasets/coco/annotations/panoptic_val2017.zip


# !mkdir -p /content/datasets/coco_panoptic
# !unzip -d /content/datasets/coco_panoptic/ /content/datasets/coco/annotations/panoptic_train2017.zip
# !unzip -d /content/datasets/coco_panoptic/ /content/datasets/coco/annotations/panoptic_val2017.zip


流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/datasets/coco/panoptic_val2017/000000353027.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000489014.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000201775.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000245651.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000446117.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000215072.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000119828.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000523957.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000309467.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000090956.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000283038.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000035326.png  
  inflating: /content/datasets/coco/panoptic_val2017/000000546659.png  
  inflating: /content/datasets/coco/p

In [ ]:
# %env DETECTRON2_DATASETS=/content/datasets
# %cd /content/
# !python ./detectron2/datasets/prepare_panoptic_fpn.py

env: DETECTRON2_DATASETS=/content/datasets
/content
Start writing to /content/datasets/coco/panoptic_stuff_val2017 ...
Finished. time: 46.18s
Start writing to /content/datasets/coco/panoptic_stuff_train2017 ...
Finished. time: 1462.56s
panoptic_val2017_100.json: 172kB [00:01, 106kB/s]               
Creating /content/datasets/coco/panoptic_val2017_100 ...
Creating /content/datasets/coco/panoptic_stuff_val2017_100 ...


# Evaluation

## DETR R50

In [11]:
!python main.py \
    --batch_size 1 --no_aux_loss --eval \
    --resume https://dl.fbaipublicfiles.com/detr/detr-r50-panoptic-00ce5173.pth \
    --masks --dataset_file coco_panoptic \
    --coco_path /content/datasets/coco/ \
    --coco_panoptic_path /content/datasets/coco_panoptic/

Not using distributed mode
git:
  sha: 8a144f83a287f4d3fece4acdf073f387c5af387d, status: clean, branch: main

Namespace(aux_loss=False, backbone='resnet50', batch_size=1, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path='/content/datasets/coco_panoptic/', coco_path='/content/datasets/coco/', dataset_file='coco_panoptic', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=True, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=True, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='https://dl.fbaipublicfiles.com/detr/detr-r50-panoptic-00ce5173.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
/usr/local/lib/python3.8/dist-packages/torc

## DETR-R50 DC5

In [12]:
!python main.py \
    --dilation \
    --batch_size 1 --no_aux_loss --eval \
    --resume https://dl.fbaipublicfiles.com/detr/detr-r50-dc5-panoptic-da08f1b1.pth \
    --masks --dataset_file coco_panoptic \
    --coco_path /content/datasets/coco/ \
    --coco_panoptic_path /content/datasets/coco_panoptic/

Not using distributed mode
git:
  sha: 8a144f83a287f4d3fece4acdf073f387c5af387d, status: clean, branch: main

Namespace(aux_loss=False, backbone='resnet50', batch_size=1, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path='/content/datasets/coco_panoptic/', coco_path='/content/datasets/coco/', dataset_file='coco_panoptic', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=True, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=True, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=True, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='https://dl.fbaipublicfiles.com/detr/detr-r50-dc5-panoptic-da08f1b1.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
/usr/local/lib/python3.8/dist-packages/t

## DETR R101 Panoptic

In [13]:
%cd /content/detr

!python main.py \
    --backbone resnet101 \
    --batch_size 1 --no_aux_loss --eval \
    --resume https://dl.fbaipublicfiles.com/detr/detr-r101-panoptic-40021d53.pth \
    --masks --dataset_file coco_panoptic \
    --coco_path /content/datasets/coco/ \
    --coco_panoptic_path /content/datasets/coco_panoptic/

/content/detr
Not using distributed mode
git:
  sha: 8a144f83a287f4d3fece4acdf073f387c5af387d, status: clean, branch: main

Namespace(aux_loss=False, backbone='resnet101', batch_size=1, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path='/content/datasets/coco_panoptic/', coco_path='/content/datasets/coco/', dataset_file='coco_panoptic', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=True, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=True, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='https://dl.fbaipublicfiles.com/detr/detr-r101-panoptic-40021d53.pth', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
/usr/local/lib/python3.8/di